# 图像增强

图像增强（Image Emhancement）指对图像的亮度、对比度、色彩等属性进行调节，以改善视觉效果，便于人眼观察的相关技术和方法的统称。

但是值得注意的是，那些面向确定的、原因已知的图像退化的改善方法、去掉运动模糊或椒盐噪声、人为修改特定结构特征的方法，一般不认为是图像增强的范畴。

常用的图像增强算法主要分为基于点运算的方法和基于统计特性的方法。

## 基于点运算的方法

基于点运算得分方法实际上是按照一定的规则逐点改变像素的信号强度。先来观察这样一幅图像：

In [ ]:
from imgtools.utils import ASSETS_DIR
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

imat_0 = np.array(Image.open(ASSETS_DIR / '01_low-contrast-pollen.png'))

print(f'Max pixel: {np.max(imat_0)}, Min pixel: {np.min(imat_0)}')

可以看到，它最大的像素值为138，最小的为91。但是一般灰度图像的像素值范围可取$[0, 255]$，所以，可以尝试把最大像素值都放大至255，或者把最小的像素值减小至0，观察图像发生的变化:

- 像素值增大，也就意味着亮度提升，使得原本暗处的细节更为明显，但是图像的对比度下降
- 像素值减小，亮度减弱，暗处的细节会更不清晰，但是主观上感觉对比度会提高，不同区域的对比和层次更明显

In [ ]:
from imgtools.operations import histogram, imshow
imshow(imat_0, 2, 3, 1, title='original')
imshow(imat_0 + 117, 2, 3, 2, title='max = 255')
imshow(imat_0 - 91, 2, 3, 3, title='min = 0')
histogram(imat_0, 2, 3, 4)
histogram(imat_0 + 117, 2, 3, 5)
histogram(imat_0 - 91, 2, 3, 6)

结合上面的思考，应该作两方面的考虑，即寻找一种方法，既能降低暗处的像素值，又可以提高亮处的像素值，兼顾亮度和对比度。考虑到基于点运算的方法，变换的输入和输出仅与像素值有关，而与像素值的空域位置无关，因此可以考虑简单的函数变换。

### 线性变换

线性变换可以表示为

$$
s = T(r) = kr + b \tag1
$$

为了最大限度实现上文的想法，预设了像素值的目标范围是$[s_1, s_2]$，因此可以简单求解出直线的斜率$k$和截距$b$（假设原图像的像素值范围是$[r_1, r_2]$）：

$$
\begin{cases}
k = \frac {s_2 - s_1} {r_2 - r_1} \\
b = s_1 - kr_1
\end{cases} \tag2
$$

代入上式即可得到下面的代码：

In [ ]:
r1, r2 = np.min(imat_0), np.max(imat_0)
s1, s2 = 0, 255

k = (s2 - s1) / (r2 - r1)
b = s1 - k * r1

def transform_l1(x):
    return k * x + b

tmp = transform_l1(imat_0)
imat_linear_transformed = np.array(tmp, dtype=np.uint8)

r = np.linspace(r1, r2, r2 - r1 + 1)
s = transform_l1(r)

plt.subplot(2, 3, (3, 6))
plt.plot(r, s)
plt.title('transformation')
imshow(imat_0, 2, 3, 1, title='original')
imshow(imat_linear_transformed, 2, 3, 2, title='transformed')
histogram(imat_0, 2, 3, 4)
histogram(imat_linear_transformed, 2, 3, 5)

显然，经过了线性变换之后的图像相对原图而言：

 - 明亮的部分更加明亮，暗淡的部分更加暗淡
 - 图像的对比度提升

上面的情况显然是$k > 1$的情形，不难想象：

 - $k = 1，改变$b$的值仅会使变换后的图像的灰度值整体上移或者下移，这只会使图像变得更亮或者更暗（参见前例）
 - $0 < k < 1$，会使得变换后图像的灰度值范围进一步缩窄，图像的黑白反差减弱
 - $k < 1$，一般而言不会出现这种变换，像素值必须是非负的
    - 但是也有特例：对图像取反色，此时图像的对比度的变化应该单独分析

但是线性变换方法很容易受到像素值的干扰，试想，假设原图像中含有值为$0$或$255$的像素值，那么按照式$(2)$求得的变换公式应为$s = T(r) = r$，此时线性变换失效。

### 对数变换

对数变换使用的函数形式为：

$$
s = T(r) = c\log(1 + r) \tag3
$$

其中$c$是常数，为了防止对$0$取对数，因此加1。对数曲线的特点是扩展低灰度值部分，而压缩高灰度值部分。

常数$c$的估算常遵循以下公式：

$$
c = \frac {s_2} {\log(1 + r_2)} \tag4
$$

In [ ]:
def tranform_log(x, c: float = 1):
    epsilon = 1e-6 # trick: avoiding occurance of NaN or inf
    return np.array(c * np.log2(x + 1 + epsilon), dtype=np.uint8)

imat_1 = np.array(Image.open(ASSETS_DIR / '00_girl.png'))
imat_log2 = tranform_log(imat_1, 30) # calulated from formula (4)

imshow(imat_1, 2, 2, 1, title = 'original')
imshow(imat_log2, 2, 2, 2, title='transformed')
histogram(imat_1, 2, 2, 3)
histogram(imat_log2, 2, 2, 4)

很显然，在进行对数变换后，图像发生的变化如下：

 - 图像整体的亮度提高，暗色区域的细节变得清晰
 - 左侧的灰度区间被拓宽，而右侧的灰度区间被收窄
 - 图像整体的对比度下降（使用Michelson对比度进行估算）

所以，对数变换适用于图像整体较暗，且需要增强图像中暗色区域细节的情况。

### 指数变换

指数变换使用的函数形式：

$$
s = c \cdot r^\gamma \tag5
$$

其中，$c$和$\gamma$都是常数。常被用于扩展图像特定的灰度范围，以及调节图像的对比度。其中$\gamma$在变换中起决定性作用，如图：

![gamma_transformation.png](https://s2.loli.net/2023/06/23/XluFwMPKE8Tf4IJ.png)

下面以`imat_0`为基础，构建一个图像灰度范围狭窄，曝光过度的灰度图像，并作指数变换，观察图像的变化。可以看到当$\gamma = 1.25 \ gt 1$时，图像的对比度提升明显，细节突出，有效的改善了图像的质量。

指数变换常用于灰度范围狭窄且曝光过度的图像的增强。

In [ ]:
imat_2 = imat_0 + 52

def tranform_pow(x, exp: float, c: float = 1):
    return np.array(c * x ** exp, dtype=np.uint8)

imat_g1 = tranform_pow(imat_2, 1.25)
imat_g2 = tranform_pow(imat_2, 0.85)

imshow(imat_2, 2, 3, 1, title='original')
imshow(imat_g1, 2, 3, 2, title=r'$\gamma = 1.25$')
imshow(imat_g2, 2, 3, 3, title=r'$\gamma = 0.85$')
histogram(imat_2, 2, 3, 4)
histogram(imat_g1, 2, 3, 5)
histogram(imat_g2, 2, 3, 6)

### 基于统计特性的方法

通过前面的一系列的例子，可以发现，主观上认为观感较好、对比度较高的图片，其直方图应该是相对均匀的分布在所有灰度值范围内，而非集中在某一个小的字区间内。那么，是不是可以通过改变其直方图的分布，从而进行图像增强呢？答案是肯定的。

